# K-Means Clustering: Mall Customer Segmentation

This notebook implements the full K-means segmentation pipeline for the Mall Customers dataset.

Objectives:
- Perform EDA and preprocessing
- Determine optimal number of clusters (k)
- Train K-means and validate clusters
- Visualize and profile segments
- Provide actionable business recommendations

Dataset: `Mall_Customers.csv` (200 rows, 5 columns)

In [ ]:
# Imports & Setup
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA

# Plot settings
sns.set(style="whitegrid", context="notebook")
plt.rcParams["figure.figsize"] = (8, 5)

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Data path
DATA_PATH = Path("Mall_Customers.csv")
assert DATA_PATH.exists(), f"Dataset not found at {DATA_PATH}. Please place Mall_Customers.csv in the same folder as this notebook."

In [ ]:
# Outputs setup
from pathlib import Path
OUTPUTS_DIR = Path("outputs")
OUTPUTS_DIR.mkdir(exist_ok=True)
print(f"Outputs will be saved to: {OUTPUTS_DIR.resolve()}")

In [ ]:
# Load data
raw = pd.read_csv(DATA_PATH)
print(raw.shape)
raw.head()

In [ ]:
# Initial EDA: structure and missing
raw.info()
print("\nMissing values per column:\n", raw.isna().sum())
print("\nDuplicate rows:", raw.duplicated().sum())

In [ ]:
# Summary statistics for numeric columns
raw.describe(include='all')

In [ ]:
# EDA: distributions for numeric features
num_cols = [c for c in raw.columns if raw[c].dtype != 'object' and c != 'CustomerID']
fig, axes = plt.subplots(1, len(num_cols), figsize=(5*len(num_cols), 4))
if len(num_cols) == 1:
    axes = [axes]
for ax, col in zip(axes, num_cols):
    sns.histplot(raw[col], kde=True, ax=ax, color="#4C78A8")
    ax.set_title(f"Distribution: {col}")
plt.tight_layout()
plt.show()

In [ ]:
# Gender balance and encoding plan
if 'Gender' in raw.columns:
    display(raw['Gender'].value_counts())
else:
    print("Gender column not found; proceeding without it.")

In [ ]:
# Correlation heatmap for numeric features
corr = raw[num_cols].corr(numeric_only=True)
plt.figure(figsize=(6,4))
sns.heatmap(corr, annot=True, cmap="Blues", fmt=".2f")
plt.title("Correlation Heatmap (Numeric Features)")
plt.show()

## Preprocessing
- Encode `Gender` to numeric
- Select features for clustering
- Standardize features

In [ ]:
# Encode Gender (Male=1, Female=0) if available
work = raw.copy()
if 'Gender' in work.columns:
    work['Gender'] = work['Gender'].map({'Male': 1, 'Female': 0})

# Feature selection rationale: Age, Annual Income (k$), Spending Score (1-100)
# Optionally include Gender if helpful; exclude CustomerID
feature_cols = []
for c in ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']:
    if c in work.columns:
        feature_cols.append(c)
if 'Gender' in work.columns:
    feature_cols.append('Gender')
print("Using features:", feature_cols)

X = work[feature_cols]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Scaled shape:", X_scaled.shape)

# Save scaler
import joblib
joblib.dump(scaler, OUTPUTS_DIR / 'scaler.joblib')

## Optimal K Selection
We evaluate k in 2–10 using Elbow (WCSS), Silhouette, Davies–Bouldin, and Calinski–Harabasz scores.

In [ ]:
# Compute metrics across k values
results = []
ks = list(range(2, 11))
for k in ks:
    km = KMeans(n_clusters=k, init="k-means++", n_init="auto", random_state=RANDOM_STATE)
    labels = km.fit_predict(X_scaled)
    sil = silhouette_score(X_scaled, labels)
    db = davies_bouldin_score(X_scaled, labels)
    ch = calinski_harabasz_score(X_scaled, labels)
    results.append({
        'k': k,
        'inertia': km.inertia_,
        'silhouette': sil,
        'davies_bouldin': db,
        'calinski_harabasz': ch
    })

res_df = pd.DataFrame(results)
res_df

In [ ]:
# Plot Elbow and Silhouette
fig, axes = plt.subplots(1, 3, figsize=(16,4))
axes[0].plot(res_df['k'], res_df['inertia'], marker='o')
axes[0].set_title('Elbow: Inertia vs k')
axes[0].set_xlabel('k'); axes[0].set_ylabel('Inertia (WCSS)')

axes[1].plot(res_df['k'], res_df['silhouette'], marker='o', color='#2ca02c')
axes[1].set_title('Silhouette vs k')
axes[1].set_xlabel('k'); axes[1].set_ylabel('Silhouette Score')

axes[2].plot(res_df['k'], res_df['davies_bouldin'], marker='o', color='#d62728')
axes[2].set_title('Davies-Bouldin vs k (lower is better)')
axes[2].set_xlabel('k'); axes[2].set_ylabel('DB Index')

plt.tight_layout()
plt.show()

# Candidate k suggestion: highest silhouette, elbow knee, and low DB
best_k_sil = int(res_df.loc[res_df['silhouette'].idxmax(), 'k'])
print("Best k by silhouette:", best_k_sil)

## Train Final K-Means
We will use the selected k (you may override `FINAL_K` if business constraints suggest a different choice).

In [ ]:
# Choose final k
FINAL_K = best_k_sil  # you may set e.g., 5 based on elbow/interpretability
print("Using FINAL_K =", FINAL_K)

kmeans = KMeans(n_clusters=int(FINAL_K), init="k-means++", n_init="auto", random_state=RANDOM_STATE)
labels = kmeans.fit_predict(X_scaled)
centroids_scaled = kmeans.cluster_centers_

# Evaluate final model
sil_final = silhouette_score(X_scaled, labels)
ch_final = calinski_harabasz_score(X_scaled, labels)
db_final = davies_bouldin_score(X_scaled, labels)

print({
    'silhouette': round(sil_final, 4),
    'calinski_harabasz': round(ch_final, 2),
    'davies_bouldin': round(db_final, 4),
    'inertia': round(kmeans.inertia_, 2)
})

# Save model
import joblib
joblib.dump(kmeans, OUTPUTS_DIR / f'kmeans_k{FINAL_K}.joblib')

In [ ]:
# Attach labels and inverse-transform centroids for interpretation
work_labeled = work.copy()
work_labeled['Cluster'] = labels

centroids = pd.DataFrame(scaler.inverse_transform(centroids_scaled), columns=feature_cols)
centroids['Cluster'] = range(int(FINAL_K))

# Cluster sizes
cluster_sizes = work_labeled['Cluster'].value_counts().sort_index()
print("Cluster sizes:\n", cluster_sizes)

# Save outputs
work_labeled.to_csv(OUTPUTS_DIR / 'customers_with_clusters.csv', index=False)
centroids.to_csv(OUTPUTS_DIR / 'cluster_centroids.csv', index=False)
centroids

## Visualization
- 2D scatterplots on key feature pairs
- PCA 2D and optional 3D
- Box plots per feature by cluster

In [ ]:
# 2D scatterplots on selected feature pairs
pairs = []
for a in ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']:
    for b in ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']:
        if a in work_labeled.columns and b in work_labeled.columns and a < b:
            pairs.append((a,b))

palette = sns.color_palette("tab10", int(FINAL_K))
for a, b in pairs:
    plt.figure(figsize=(6,4))
    sns.scatterplot(data=work_labeled, x=a, y=b, hue='Cluster', palette=palette, alpha=0.8)
    # plot centroids
    if a in centroids.columns and b in centroids.columns:
        plt.scatter(centroids[a], centroids[b], s=200, c='black', marker='X', label='Centroid')
    plt.title(f"Clusters on {a} vs {b}")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# PCA 2D (and optional 3D)
pca2 = PCA(n_components=2, random_state=RANDOM_STATE)
X_pca2 = pca2.fit_transform(X_scaled)

plt.figure(figsize=(7,5))
sns.scatterplot(x=X_pca2[:,0], y=X_pca2[:,1], hue=labels, palette=palette, alpha=0.8)
centroids_pca2 = pca2.transform(centroids_scaled)
plt.scatter(centroids_pca2[:,0], centroids_pca2[:,1], s=200, c='black', marker='X', label='Centroid')
plt.title("PCA 2D Projection of Clusters")
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()

In [ ]:
# Box plots per feature by cluster
for col in feature_cols:
    plt.figure(figsize=(6,4))
    sns.boxplot(data=work_labeled, x='Cluster', y=col, palette=palette)
    plt.title(f"{col} by Cluster")
    plt.tight_layout()
    plt.show()

## Cluster Profiling & Business Insights
We summarize per-cluster statistics and create segment personas.

In [ ]:
# Per-cluster summary stats
profile = work_labeled.groupby('Cluster')[feature_cols].agg(['mean','median','count']).round(2)
profile

## Recommendations
Concrete actions per segment based on centroid interpretation and profiles.

In [ ]:
# Draft recommendation framework based on profiles
segments = []
for cid, row in centroids.sort_values('Cluster').iterrows():
    desc = {
        'Cluster': int(row['Cluster']),
        'Persona': None,
        'Message': None,
        'Offers': None,
        'Channels': None
    }
    age = row.get('Age', np.nan)
    inc = row.get('Annual Income (k$)', np.nan)
    score = row.get('Spending Score (1-100)', np.nan)

    if pd.notna(score) and score >= np.nanpercentile(centroids['Spending Score (1-100)'], 60):
        desc['Persona'] = 'High-spend Enthusiasts'
        desc['Message'] = 'Exclusive previews and new arrivals'
        desc['Offers'] = 'VIP tiers, early access, bundles'
        desc['Channels'] = 'App push, email, in-store concierge'
    elif pd.notna(inc) and inc >= np.nanpercentile(centroids['Annual Income (k$)'], 60):
        desc['Persona'] = 'Affluent Occasional Buyers'
        desc['Message'] = 'Quality and premium experiences'
        desc['Offers'] = 'Premium brand bundles, white-glove service'
        desc['Channels'] = 'Email, social retargeting, SMS reminders'
    else:
        desc['Persona'] = 'Value Seekers'
        desc['Message'] = 'Best value for money'
        desc['Offers'] = 'Discounts, loyalty points, price-match'
        desc['Channels'] = 'SMS, email, in-app coupons'

    segments.append(desc)

rec_df = pd.DataFrame(segments)
rec_df

## Final Notes
- Chosen k justification references elbow, silhouette, and DB indices.
- Save artifacts and outputs for reproducibility.

In [ ]:
# Optional: PCA 3D projection
try:
    from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
    pca3 = PCA(n_components=3, random_state=RANDOM_STATE)
    X_pca3 = pca3.fit_transform(X_scaled)

    fig = plt.figure(figsize=(7,6))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(X_pca3[:,0], X_pca3[:,1], X_pca3[:,2], c=labels, cmap='tab10', alpha=0.8)
    centroids_pca3 = pca3.transform(centroids_scaled)
    ax.scatter(centroids_pca3[:,0], centroids_pca3[:,1], centroids_pca3[:,2], c='black', s=200, marker='X')
    ax.set_title('PCA 3D Projection of Clusters')
    plt.tight_layout()
    plt.show()
except Exception as e:
    print('3D plot could not be generated:', e)

In [ ]:
# Utility display import for EDA outputs
from IPython.display import display

In [ ]:
# Box/Violin plots by cluster for each feature
plot_features = [c for c in ['Age','Annual Income (k$)','Spending Score (1-100)','Gender'] if c in work_labeled.columns]
for col in plot_features:
    plt.figure(figsize=(6,4))
    sns.boxplot(data=work_labeled, x='Cluster', y=col, palette="tab10")
    plt.title(f"Boxplot of {col} by Cluster")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(6,4))
    sns.violinplot(data=work_labeled, x='Cluster', y=col, palette="tab10", inner='quartile')
    plt.title(f"Violin plot of {col} by Cluster")
    plt.tight_layout()
    plt.show()